In [ ]:
# Data Cleaning:
# 1. [Done] Data cleaning: Take a closer look at text with keyword "平台"
# 2. Clean timestamp
# 3. [Done] More data cleaning: combine title and description only when title != description
# 4. HanLP summarize topics and keywords--use this to filter out nonrelevant documents
# 5. [Done] Deal with links in descriptions
# 6. [Done] Deal with noises (e.g. \n)


# To do:
# 1. [Done] Keywords to check: "新能源" 
# 2. [Done]'江淮','轻卡', '中卡', '重卡', '冷藏车'，'解放', '购车', '重汽', '牵引车', '厢式货车','轮胎', '发动机', '欧曼'
# # # check these keywords again
# # # 以上关键词下有大量无用信息，但是也有很多涉及到‘货拉拉’等平台的讨论。比如跑货拉拉买什么车好。这个问题说明司机为了平台专门买车。
# # # 所以这几个词不能全部drop，要进一步考虑如何filter掉仅仅讨论车的内容。
# # # 另外可以去除涉及到这几个关键词的长度过长的帖子。
# # # 有没有自动判定是广告的package？
# 3. [Done] Check pattern: '拉货|货源|网络平台|跑平台|找货|平台接单|货运配送信息平台|货运平台|走平台|APP|货运信息发布平台|货运APP|数字化货运'
# 4. [Done] Check pattern = r'(?=.*卡车之家)(?=.*平台)(?=.*感谢)' 
# 5. [Done] 检查平台名称的昵称，如货**，祸啦啦，yunmanman, ‘货 车 帮’etc
# 6. [Done] join title and description when title != description
# 7. [Done] Deal with links in 'description'
# 8. Retrieve from drop_df  if keyword != 平台
# 9. [Done] Define as safe_data if title includes keywords but description is empty
# 10. [Done] fix new line (\r\n) 
# 11. [Done] Fix other noises, e.g. #, extra space  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [ ]:
df = pd.read_parquet('RawData.parquet')

In [ ]:
# Check if 'tid' has duplicated values
df['tid'] = pd.to_numeric(df['tid'])
df['tid'].value_counts()

In [ ]:
# drop duplicates
df = df.drop_duplicates(subset=['tid'])

In [ ]:
# Drop rows containing "直播平台" and more 
dropped_keywords = r'直播平台|亚马逊平台|卖车平台|Temu平台'  
contains_dropped_keywords = df['title'].str.contains(dropped_keywords, case=False, na=False)|df['description'].str.contains(dropped_keywords, case=False, na=False)
df = df[~contains_dropped_keywords]
df.info()

In [ ]:
df = df.drop('isHot', axis=1)

In [ ]:
# Deal with links in 'description'

# Regular expression pattern for matching URLs
url_pattern = r'(http[s]?://[^\s]+)'
def extract_urls(x):
    return re.findall(url_pattern, x)

df['links'] = df['description'].apply(extract_urls)

# Remove URLs from the original column
df['description'] = df['description'].str.replace(url_pattern, '', regex=True)

df.info()

In [ ]:
# Drop new line signs 
df['description'] = df['description'].replace(r'\r\n|\r|\n', '', regex=True)
print(df['description'].head(20))

In [ ]:
# Drop noises (e.g. #)
df['description'] = df['description'].replace(r'#', '', regex=True)
df['title'] = df['title'].replace(r'#', '', regex=True)

# Remove all spaces
df['description'] = df['description'].str.replace(' ', '', regex=True)
df['title'] = df['title'].str.replace(' ', '', regex=True)

In [ ]:
# combine title and description into full text when text != description

def join_des_title(row):
    title = row['title']
    des = row['description']
    if title != des:
        fulltext = title + des
    else:  
        fulltext = title
    return fulltext  # Explicitly return the combined value.

# Apply the function across each row (axis=1)
df['fulltext'] = df.apply(join_des_title, axis=1)

df.info()

In [ ]:
# These are rows that do not need to drop for certain
safe_words = r'运满满|货车帮|满帮|货拉拉'
safe_data = df[df['fulltext'].str.contains(safe_words)]
safe_data.info()

In [ ]:
# Create drop_df to store hand-labeled data
drop_rows = [4206269, 4189528, 4199761, 4198174, 4158945, 4160055, 4156021, 4180746, 4153232, 4150274,
            4140355, 4144839, 4140023, 4177614, 4140098, 4140661, 4118098, 4198331, 4112225, 4155607,
            4116943, 4127864, 4154871, 4230073, 4223087, 4147394, 4092950, 4225032, 4224797, 4201612, 
            4221406, 4230074, 4094530, 4208279, 4169342, 4224947, 4221507, 4218582, 4186372, 4138462, 
            4160339, 4152716, 4167395, 4131743, 4184764, 4135173, 4097525, 
            4225765, 4062857, 4194521, 4062857, 4021542, 4103677, 3912846,3940659,4106426,3843810,4019347,4063065,4005551,4009488,
             3955223, 3958849, 4006490, 3948245, 3795755, 3723194, 3879128, 3882277, 3871359, 3999604, 4021166, 3993411, 3876568,
             4019496, 3951974, 3937517, 3936127, 3924264, 3953073, 3813832, 3817042, 3857828, 3791837, 3791837, 3634110, 3787212,
             3909370, 3798534, 3700722, 3853733, 3870739, 3860417, 3705489, 3724751, 3793107, 3732709, 3820384, 3799585, 3465302, 
             3779986, 3723896, 3680179, 3715496, 3654687, 3782826, 3667343, 3670672, 3640110, 3662713, 3681488, 3653932, 3458447,
             3617418, 3622289, 3276495, 3063478, 3514443, 3412827, 3453601, 3501039, 3445889, 3567600, 3612575, 3388163, 3346517,
             3599728, 3285466, 3394530, 3284597, 3254212, 2566325, 3178290, 3146624, 3166987, 2598661, 3125868, 3339152, 3342357,
             3083526, 3054823, 2478794, 3134632, 2914100, 3098460, 3143003, 2881666, 2929605, 3122733, 2721389, 2891674, 3184403,
             2956741, 2727408, 3037502, 2549572, 4138813, 4147394, 4136990, 4138462, 4160339, 4152716, 4167395, 4131743, 4184764, 
             4135173, 4138814, 4097525, 4076294, 4139143, 4074414, 4119061, 4032570, 4063539, 4046712]  # manually selected with keyword '卡车之家' and '平台'

drop_df = df[df['tid'].isin(drop_rows)]
# drop_df.to_excel('dropped_data.xlsx')

In [ ]:
# identify authors to drop
drop_authors = ['淡定人生jewv', '大力士', '辽宁卡友zgqkte', '重庆卡友lrezaq', '卡家小林', '卡家木木', '卡家小白兔', '卡家小双',
                '江铃轻卡家族', '山东卡友xmwvlo', '山东卡友dahyjv', '山东卡友cqtwzm', '山东卡友zqhydb', '卡家小太阳', '卡家大鹏', '卡家短视频', 
                '卡家小新', '卡家小官', '卡家大白', '卡家商城—大白', '卡家小肉丸', '卡家小贝壳', '卡家运力', '卡家小编', '卡家约稿君', 
                '卡家二手车', '卡家大鑫', '卡家小威', '卡家小米', 'kayouLlvL', '⒉冄1⒋号', '卡车之家商城'] 
drop_df = pd.concat([drop_df, df[df['author'].isin(drop_authors)]])

# '卡家小白兔' keep 'tid' in [4078138, 4071625, 4073974, 4073597, 4068177, 4050297], but need to re-collect content data

In [ ]:
drop_df.info()

In [ ]:
# Remove these rows of safe_data and drop_df to simplify clean_data a bit
uncertain_data = pd.DataFrame()
for i in df['tid']:
    if i not in (list(drop_df['tid']) + list(safe_data['tid'])):
        uncertain_data = pd.concat([uncertain_data, df[df['tid'] == i]])

uncertain_data.info()

In [ ]:
drop_keywrds = r'急聘|卡车之家商城|抖音平台'
drop_df = pd.concat([drop_df, uncertain_data[uncertain_data['fulltext'].str.contains(drop_keywrds, regex=True)]])

In [ ]:
test_1 = r'(?=.*卡车之家)(?=.*平台)(?=.*感谢)'
drop_df = pd.concat([drop_df, uncertain_data[uncertain_data['description'].str.contains(test_1, regex=True, flags=re.DOTALL)]])
drop_df.info()

In [ ]:
# # Here are all the patterns that can be dropped:
test_0 = r'(?=.*(?:江淮|购车|欧曼|一汽解放|重汽))(?=.*(?:重卡|轻卡|中卡|冷藏车|牵引车|厢式货车|轮胎|发动机|新能源))'
drop_df = pd.concat([drop_df, uncertain_data[uncertain_data['description'].str.contains(test_0, regex=True, flags=re.DOTALL)]])

In [ ]:
# drop all '江淮' except for which containing words like '拉货|货源|网络平台|跑平台|找货'
include_words = r'江淮'
exclude_words = r'跑平台|跑货车'

# Select rows that contain the inclusion keyword and do not contain the exclusion keyword
filtered_rows = uncertain_data[(uncertain_data['description'].str.contains(include_words)) & (~uncertain_data['description'].str.contains(exclude_words))]

drop_df = pd.concat([drop_df, filtered_rows])

In [ ]:
# select rows that contain at least two of the above keywords
strings_to_search = ['设备', '东风', '江铃', '江淮', '购车', '欧曼', '一汽解放', '重汽', '新能源', '重卡', '轻卡', '中卡', 
                     '冷藏车', '牵引车', '厢式货车', '轮胎', '发动机']

# Count occurrences and filter rows
uncertain_data['match_count'] = uncertain_data['description'].apply(lambda x: len([match for match in strings_to_search if match in x]))
filtered_df = uncertain_data[uncertain_data['match_count'] >= 2]

# print(len(filtered_df['description']))
# print(filtered_df['description'])

drop_df = pd.concat([drop_df, filtered_df])

In [ ]:
# test 拉货|货源|网络平台|跑平台|找货|平台接单|货运配送信息平台|货运平台|走平台|APP|货运信息发布平台|货运APP|数字化货运

safe_words = safe_words + r'数字化货运|货运APP|走平台|货运平台|平台接单|找货|跑平台|网络平台|货源|拉货'
good_text = uncertain_data[uncertain_data['fulltext'].str.contains(safe_words, regex=True)]
safe_data = pd.concat([safe_data, good_text])
safe_data.info()

In [ ]:
safe_title = r'运费冻结|平台作梗|试点平台|货运平台|找货平台|车货匹配平台'
good_title = uncertain_data['title'].str.contains(safe_title, regex=True)
desc_empty = uncertain_data['description']==''

safe_data = pd.concat([safe_data, uncertain_data[desc_empty & good_title]])
drop_df = pd.concat([drop_df, uncertain_data[desc_empty & (~good_title)]])
print(drop_df.info())
print(safe_data.info())

In [ ]:
safe_ctgry = uncertain_data[uncertain_data['keyword']=='满帮']
safe_data = pd.concat([safe_data, safe_ctgry])

In [ ]:
drop_ctgry = uncertain_data[(uncertain_data['keyword']=='货车帮')|(uncertain_data['keyword']=='货拉拉')]
drop_df = pd.concat([drop_df, drop_ctgry])

In [ ]:
sorted(uncertain_data['topicName'].unique())

In [ ]:
safe_topic = ['下半年运价有望好转吗？',  '九月发帖不停，赢车模、卫衣', '买新车的来报到', '今年的运费咋样了？', '低价货拉还是不拉？',
              '你对货运平台的看法', '你对货运环境都有哪些意见?',  '你用过哪些网络货运平台？', '你见过哪些新型货运骗局？',
              '你那里运费上涨了吗？',  '冬季行车需要注意些什么？',  '卡友有话说（我为行业发声）', '卡友闲暇时光秀', '如何看待“货车超载入刑”？', 
              '平台冻结运费合理吗？', '庆中秋共团圆',  '我在行业这些年',  '我对2021货运行业的看法',  '我对货运平台的看法',  '我的拉散货经历', 
              '扒一扒货运平台的套路', '晒油耗，谈省油', '疫情时代，各地的运费涨了吗？', '祥菱大熊猫的动力有多大？',  '给新手司机的经验之谈', 
              '老司机平常找货的渠道有哪些？', '老赖货主扣运费咋办？',  '货拉拉陷“停运”风波', '货运平台计价规则公开？', '购车应该规避哪些坑？',
              '运费一降再降，你怎么看', '香菱角的致富理想型' ]

safe_topic_data = uncertain_data[uncertain_data['topicName'].isin(safe_topic)]
safe_data = pd.concat([safe_data, safe_topic_data])

In [ ]:
bad_topic = [' 疫情下的卡车司机', '2023年定个小目标', '38妇女节·爱在卡嫂', '618晒单赢总成、冰箱、坐垫', '【迷惑】新能源选车攻略',
 '你对江淮德沃斯中卡的真感受', '你期待柔性执法吗？', '你见过新能源有啥政策？', '信息广场', '发帖七天乐，赢车模、定制雨衣',
 '发帖有礼：挑战8月榜单', '哪里行程码带星？要通行证？', '喜迎新春7天乐', '我的第一辆车是，怎么买的它？', '拍出我的货运路',
 '新手上路注意事项', '晒出你的卡家好物', '晒出我拉的货', '晒晒我的驾驶室', '欢度国庆，献礼73周年', '正在发生的事', '每日发帖赢好礼',
 '每日维修保养小记', '说说你认为值得买的车', '谈谈你对LNG卡车的了解', '货车维修被坑记', '货运人的端午节', '货运市场会迎来春天吗？',
 '那些年，我开过的解放车', '重卡+玉柴为啥这么省油', '金秋收获季，发帖赢冲锋衣', '领航卡车·美好生活']
bad_topic_data = uncertain_data[uncertain_data['topicName'].isin(bad_topic)]
drop_df = pd.concat([drop_df, bad_topic_data])

In [ ]:
# drop_words = "江淮|购车|欧曼|一汽解放|重汽|新能源|重卡|轻卡|中卡|冷藏车|牵引车|厢式货车|轮胎|发动机"
drop_word_title = r'展览|博览会|国际|设备|自卸车|底盘|抑尘车|江铃|江淮|欧曼|三一江山|东风|金杯|洒水车|NQI|侧翻|重汽|重卡|中卡|轻卡|牵引车|极氪|法士特|福田|长安|升降平台|新朱宏|骏铃|新能源|支撑平台|租赁平台'
drop_df = pd.concat([drop_df, uncertain_data[uncertain_data['title'].str.contains(drop_word_title, regex=True)]])

In [ ]:
drop_keywrds = r'欧曼|一汽解放|重汽|重卡|中卡|轻卡|牵引车|洒水车|东风|侧翻|法士特|福田|长安|升降平台|骏铃|支撑平台|租赁平台|抑尘车|底盘|自卸车'
drop_df = pd.concat([drop_df, uncertain_data[uncertain_data['description'].str.contains(drop_keywrds, regex=True)]])

In [ ]:
long_text = uncertain_data[uncertain_data['description'].apply(lambda x: len(x) > 1000)]
drop_df = pd.concat([drop_df, long_text])

In [ ]:
# updata uncertain_data--run this every time after updating drop_df
uncertain_data = pd.DataFrame()
for i in df['tid']:
    if i not in (list(drop_df['tid']) + list(safe_data['tid'])):
        uncertain_data = pd.concat([uncertain_data, df[df['tid'] == i]])

In [ ]:
# remove duplicates
uncertain_data = uncertain_data.drop_duplicates(subset=['tid'])
safe_data = safe_data.drop_duplicates(subset=['tid'])
drop_df = drop_df.drop_duplicates(subset=['tid'])

print(len(uncertain_data['tid']))
print(len(safe_data['tid']))
print(len(drop_df['tid']))

# More cleaning with GPT results

In [ ]:
# Load in chatgpt classified data
import json
from pathlib import Path
fpath = Path('data-cleaning-uncertain-by-chatgpt.json')
gpt_data = json.loads(fpath.read_text())
gpt_data

In [ ]:
gpt_df = pd.DataFrame.from_dict(gpt_data, orient='index', columns=['reason', 'result'])

In [ ]:
gpt_df.reset_index(inplace=True)
gpt_df.rename(columns={'index': 'tid'}, inplace=True)

In [ ]:
gpt_df['tid'] = pd.to_numeric(gpt_df['tid'])
gpt_df.info()
gpt_df['tid']

In [ ]:
# Merge with safe_data if result == 1, keep the result column
# Merge with drop_data if result == 0, keep the result column

gpt_keep = pd.DataFrame()
gpt_drop = pd.DataFrame()
for i in range(len(gpt_df)):
    result = gpt_df.iloc[i]['result']
    if result == 1:
        gpt_keep = pd.concat([gpt_keep, uncertain_data[uncertain_data['tid'] == gpt_df.iloc[i]['tid']]])      
    if result == 0:
        gpt_drop = pd.concat([gpt_drop, uncertain_data[uncertain_data['tid'] == gpt_df.iloc[i]['tid']]])
        
        
print(gpt_keep.info())
print(gpt_drop.info())

safe_data = pd.concat([safe_data, gpt_keep])
drop_df = pd.concat([drop_df, gpt_drop])
print(safe_data.info())
print(drop_df.info())

In [ ]:
# updata uncertain_data--run this every time after updating drop_df
uncertain_data = pd.DataFrame()
for i in df['tid']:
    if i not in (list(drop_df['tid']) + list(safe_data['tid'])):
        uncertain_data = pd.concat([uncertain_data, df[df['tid'] == i]])

In [ ]:
safe_data.info()

In [ ]:
safe_data = safe_data.drop(['match_count'], axis=1)

In [ ]:
# Save safe_data as clean_data_final
safe_data.to_parquet('clean_data_final.parquet')
drop_df.to_parquet('dropped_data_final.parquet')